<p style="text-align:center">
PSY 381D <b>Brain Connectivity</b>, Spring 2019


<img style="width: 700px; padding: 0px;" src="https://github.com/sathayas/JupyterConnectivitySpring2019/blob/master/Images/Banner.png?raw=true" alt="title pics"/>

</p>

<p style="text-align:center; font-size:40px; margin-bottom: 30px;"><b> Brain atlases & node definition </b></p>

<p style="text-align:center; font-size:18px; margin-bottom: 32px;"><b>March 11, 2019</b></p>

<hr style="height:5px;border:none" />

# 1. Data
<hr style="height:1px;border:none" />

  

* Data (explain atlases)
* Header info (voxel size, dim)
* Image data
* Reslicing the atlas
* forming a network
  * Extracting mean time course
  * Correlation matrix
  * Thresholding
* Network characteristics
  * Small-worldness
  * Centrality
  * Modularity
 